In [1]:
import serial
import numpy as np
from ParseSerial import ReadSentence_Gyro
import requests
import time
import sys
import h5py
import os
import glob


In [2]:
#Starting with Influx 2.0, authorization tokens are necessary to access databases
#The token can be accessed under the "Data" tab in the Influx UI.

#Stringing together query access
INFLUX_TOKEN='AjsrNgY_k97FMvgfCsgc2tPTx-lOVM-aYaCMjymNVIWpoSCkYh7H4AqIV9pLQHHk07zJa5pxTn4lo-3Ashwu5Q=='
ORG="tim@upenn"
INFLUX_CLOUD_URL='localhost'
BUCKET_NAME='sensors'

#Can change precision if needed
QUERY_URI='http://{}:8086/api/v2/write?org={}&bucket={}&precision=ns'.format(INFLUX_CLOUD_URL,ORG,BUCKET_NAME)

headers = {}
headers['Authorization'] = 'Token {}'.format(INFLUX_TOKEN)

#Line protocol
location_tag = 'lab'
measurement_name = 'gyro_data'

data_end_time = int(round(time.time() * 1000000000))


In [3]:
# connect to gyro
port = '/dev/ttyUSB1'
baudrate = 115200
gyro = serial.Serial(port=port, baudrate=baudrate)

In [4]:
header_gyro = "fe81ff55"
splitline_gyro = [8, 16, 24, 32, 40, 48, 50, 52, 56]
sentence_length_gyro = 72

In [5]:
#Number of data samples in each save file. Divide by 100 for seconds. (Gyroscope currently set to 100hz)
N = 60000 
dataCount = 0

In [6]:
#Creating HDF5 datasets... maybe convert to for loop
timestr = time.strftime("%Y%m%d-%H%M%S")
datestamp = time.strftime("%Y-%m-%d-Gyro")

#First check if there is a folder with current date. If there isn't, create one.
if os.path.isdir("/home/user/tim-daq/data/"+datestamp) == 0:
    os.mkdir("/home/user/tim-daq/data/"+datestamp)

#Then check if the folder is empty. If it's empty, the first file number will start at 1.
#If the folder is not empty, check the file number of the last edited file, and add 1 to it for the new file. 
if len(os.listdir("/home/user/tim-daq/data/"+datestamp)) == 0:
    file_number = 1
    file_number_string =  str(1)
    
else:
    file_list = glob.glob("/home/user/tim-daq/data/"+datestamp+"/*.hdf5")
    latest_file = max(file_list, key=os.path.getctime)
    file_number = int(latest_file[-10:-5]) + 1
    file_number_string =  str(file_number)

gyrofile = h5py.File("/home/user/tim-daq/data/"+datestamp+"/Gyro_"+timestr+"_"+file_number_string.zfill(5)+".hdf5", "w", libver = 'latest')
sn_dset = gyrofile.create_dataset("seq_num", (N,), dtype='i', chunks = True, maxshape = (None,))
x_dset = gyrofile.create_dataset("x", (N,), dtype='float32', chunks = True, maxshape = (None,))
y_dset = gyrofile.create_dataset("y", (N,), dtype='float32', chunks = True, maxshape = (None,))
z_dset = gyrofile.create_dataset("z", (N,), dtype='float32', chunks = True, maxshape = (None,))
st_dset = gyrofile.create_dataset("status", (N,), dtype='i', chunks = True, maxshape = (None,))
temp_dset = gyrofile.create_dataset("temp", (N,), dtype='float', chunks = True, maxshape = (None,))
time_dset = gyrofile.create_dataset("time", (N,), dtype='int64', chunks = True, maxshape = (None,))
index_dset = gyrofile.create_dataset("index", (1,), dtype='int64', chunks = True, maxshape = (None,))
filenames_dset = gyrofile.create_dataset("filenames", (2,), dtype='int64', chunks = True, maxshape = (None,))

filenames_dset[0] = file_number
filenames_dset[1] = file_number + 1

gyrofile.swmr_mode = True

In [7]:
#Sending data in chunks instead of continuous stream. (To Influx)


# Increase the points, 2, 10 etc.
#number_of_points = 10
#batch_size = 5

# while True:
# # while gyro.in_waiting == 0:
# #     pass
#     for i in np.arange(number_of_points):   
#         #while len(buffer_sn) < bufferLength:
#         sn_buff, x_buff, y_buff,z_buff, st_buff, temp_buff = ReadSentence_Gyro(gyro, header_gyro, splitline_gyro, sentence_length_gyro) 
#         #print(x_buff)
#         current_point_time = int(round(time.time() * 1000000000))
#         gyro_data.append('{measurement},location={location} x={x},y={y},z={z},temp={temp} {timestamp}'
#                          .format(measurement=measurement_name,location=location_tag, x=x_buff,y=y_buff, z=z_buff,temp=temp_buff, timestamp=current_point_time))      

#     if __name__ == '__main__':
#       # Check to see if number of points factors into batch size
#         if (number_of_points % batch_size != 0 ):
#             raise SystemExit( 'Number of points must be divisible by batch size' )
#         # Newline delimit the data
#         for batch in range(0, len(gyro_data), batch_size):
#             current_batch = '\n'.join( gyro_data[batch:batch + batch_size] )
#             #print(gyro_data)
#             print(current_batch)
#             r = requests.post(QUERY_URI, data=current_batch, headers=headers)
#             #print(r.status_code)

In [ ]:
while True:
   
    sn_buff, x_buff, y_buff,z_buff, st_buff, temp_buff = ReadSentence_Gyro(gyro, header_gyro, splitline_gyro, sentence_length_gyro) 
     
    current_point_time = int(round(time.time() * 1000000000))
    directory = time.strftime("%Y-%m-%d-Gyro")
    
    current_line = '{measurement},location={location} gyro_x={x},gyro_y={y},gyro_z={z},gyro_temp={temp} {timestamp}'.format(measurement=measurement_name,location=location_tag, x=x_buff,y=y_buff, z=z_buff,temp=temp_buff, timestamp=current_point_time)       
    #print(current_line)
    r = requests.post(QUERY_URI, data=current_line, headers=headers)
    #print(r.status_code)
    
    if dataCount >= N:
        gyrofile.close()
        current_timestr = time.strftime("%Y%m%d-%H%M%S")
        datestamp = time.strftime("%Y-%m-%d-Gyro")
        
        if os.path.isdir("/home/user/tim-daq/data/"+datestamp) == 0:
            os.mkdir("/home/user/tim-daq/data/"+datestamp)  
        
        file_list = glob.glob("/home/user/tim-daq/data/"+datestamp+"/*.hdf5")
        latest_file = max(file_list, key=os.path.getctime)
        file_number = int(latest_file[-10:-5]) + 1
        file_number_string =  str(file_number)
        
        
       
        gyrofile = h5py.File("/home/user/tim-daq/data/"+datestamp+"/Gyro_"+current_timestr+'_'+file_number_string.zfill(5)+".hdf5", "w", libver = 'latest')
        sn_dset = gyrofile.create_dataset("seq_num", (N,), dtype='i', chunks = True, maxshape = (None,))
        x_dset = gyrofile.create_dataset("x", (N,), dtype='float32', chunks = True, maxshape = (None,))
        y_dset = gyrofile.create_dataset("y", (N,), dtype='float32', chunks = True, maxshape = (None,))
        z_dset = gyrofile.create_dataset("z", (N,), dtype='float32', chunks = True, maxshape = (None,))
        st_dset = gyrofile.create_dataset("status", (N,), dtype='i', chunks = True, maxshape = (None,))
        temp_dset = gyrofile.create_dataset("temp", (N,), dtype='float', chunks = True, maxshape = (None,))
        time_dset = gyrofile.create_dataset("time", (N,), dtype='int64', chunks = True, maxshape = (None,))
        index_dset = gyrofile.create_dataset("index", (1,), dtype='int64', chunks = True, maxshape = (None,))
        filenames_dset = gyrofile.create_dataset("filenames", (2,), dtype='int64', chunks = True, maxshape = (None,))

        filenames_dset[0] = file_number
        filenames_dset[1] = file_number + 1
        gyrofile.swmr_mode = True
        dataCount = 0
    else:
        
        sn_dset[dataCount] = sn_buff
        x_dset[dataCount] = x_buff
        y_dset[dataCount] = y_buff
        z_dset[dataCount] = z_buff
        st_dset[dataCount] = st_buff
        temp_dset[dataCount] = temp_buff
        time_dset[dataCount] = current_point_time
        index_dset[0] = dataCount
        
        sn_dset.flush()
        x_dset.flush()
        y_dset.flush()
        z_dset.flush()
        st_dset.flush()
        temp_dset.flush()
        index_dset.flush()
        
        dataCount += 1
    
    

In [ ]:
#for i in np.arange(N):  
    #sn_dset[i], x_dset[i], y_dset[i], z_dset[i], st_dset[i], temp_dset[i] = ReadSentence_Gyro(gyro, header_gyro, splitline_gyro, sentence_length_gyro)
    #print(seq_num, x, y, z, status, temperature)
    #print(sn_dset[i], x_dset[i], y_dset[i], z_dset[i], st_dset[i], temp_dset[i])
    